In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName("group_window")\
                    .getOrCreate()
                    


24/04/17 21:44:03 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface wlp0s20f3)
24/04/17 21:44:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/17 21:44:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.format("csv")\
                .option("header",True)\
                .option("inferschema",True)\
                .load("./sales_transactions.csv")
                
df.show()

+---+------------+------+-----------+
| ID|Product Name| Price|Total Value|
+---+------------+------+-----------+
|  1|      Jacket|872.19|    7849.71|
|  2|       Jeans|865.39|    1730.78|
|  3|         Hat|183.11|     549.33|
|  4|     Sweater|305.15|      610.3|
|  5|         Hat| 84.59|      845.9|
|  6|       Socks|191.72|    1533.76|
|  7|      Jacket|696.02|     696.02|
|  8|     Sweater|170.31|    1532.79|
|  9|       Jeans|163.04|    1141.28|
| 10|     Sweater| 990.3|     1980.6|
| 11|     Sweater| 92.88|     743.04|
| 12|       Shoes|306.33|    1531.65|
| 13|       Shirt| 65.67|     197.01|
| 14|      Jacket|555.63|    4445.04|
| 15|       Jeans| 48.99|     342.93|
| 16|      Jacket|323.87|    2590.96|
| 17|      Jacket| 895.2|     8952.0|
| 18|       Shoes|106.37|     531.85|
| 19|         Hat|146.77|     880.62|
| 20|       Socks|584.94|    5264.46|
+---+------------+------+-----------+
only showing top 20 rows



In [4]:
df1 = df.groupBy(col('Product Name')).agg(sum(col("Total Value")).alias("total_value")).orderBy(desc(col("total_value")))
df1.show()

+------------+------------------+
|Product Name|       total_value|
+------------+------------------+
|      Jacket| 378274.5700000001|
|       Jeans|362197.52999999985|
|       Socks| 299744.8399999999|
|       Dress| 299719.0699999999|
|       Shirt| 274149.0500000001|
|       Pants|         269397.16|
|         Hat|251633.72999999995|
|       Shoes|         248539.26|
|     Sweater|246963.95000000004|
|       Skirt|211751.48000000013|
+------------+------------------+



In [5]:
# window_specs = Window.orderBy(col("total_value"),desc())
window_specs = Window.orderBy(desc("total_value"))

In [6]:
window_specs = Window.orderBy(desc("total_value"))
df2 = df1.withColumn("rank", dense_rank().over(window_specs))
dff = df2.filter((col("rank") < 6) & (col("rank") > 1))
dff.show()

24/04/17 21:44:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------------+----+
|Product Name|       total_value|rank|
+------------+------------------+----+
|       Jeans|362197.52999999985|   2|
|       Socks| 299744.8399999999|   3|
|       Dress| 299719.0699999999|   4|
|       Shirt| 274149.0500000001|   5|
+------------+------------------+----+



In [7]:
df4 = df2.filter((col("rank") < 6) & (col("rank") > 2))
df4.show()

24/04/17 21:44:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+-----------------+----+
|Product Name|      total_value|rank|
+------------+-----------------+----+
|       Socks|299744.8399999999|   3|
|       Dress|299719.0699999999|   4|
|       Shirt|274149.0500000001|   5|
+------------+-----------------+----+



In [8]:
window_specs = Window.orderBy(desc("total_value"))
df2 = df1.withColumn("rank", dense_rank().over(window_specs))
df2.show()


24/04/17 21:44:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------------+----+
|Product Name|       total_value|rank|
+------------+------------------+----+
|      Jacket| 378274.5700000001|   1|
|       Jeans|362197.52999999985|   2|
|       Socks| 299744.8399999999|   3|
|       Dress| 299719.0699999999|   4|
|       Shirt| 274149.0500000001|   5|
|       Pants|         269397.16|   6|
|         Hat|251633.72999999995|   7|
|       Shoes|         248539.26|   8|
|     Sweater|246963.95000000004|   9|
|       Skirt|211751.48000000013|  10|
+------------+------------------+----+



In [9]:
df.show()

+---+------------+------+-----------+
| ID|Product Name| Price|Total Value|
+---+------------+------+-----------+
|  1|      Jacket|872.19|    7849.71|
|  2|       Jeans|865.39|    1730.78|
|  3|         Hat|183.11|     549.33|
|  4|     Sweater|305.15|      610.3|
|  5|         Hat| 84.59|      845.9|
|  6|       Socks|191.72|    1533.76|
|  7|      Jacket|696.02|     696.02|
|  8|     Sweater|170.31|    1532.79|
|  9|       Jeans|163.04|    1141.28|
| 10|     Sweater| 990.3|     1980.6|
| 11|     Sweater| 92.88|     743.04|
| 12|       Shoes|306.33|    1531.65|
| 13|       Shirt| 65.67|     197.01|
| 14|      Jacket|555.63|    4445.04|
| 15|       Jeans| 48.99|     342.93|
| 16|      Jacket|323.87|    2590.96|
| 17|      Jacket| 895.2|     8952.0|
| 18|       Shoes|106.37|     531.85|
| 19|         Hat|146.77|     880.62|
| 20|       Socks|584.94|    5264.46|
+---+------------+------+-----------+
only showing top 20 rows



In [10]:
df3 = df.withColumn("sale", concat_ws("|",col("Price"),col("Total Value")))
df3.show()

+---+------------+------+-----------+--------------+
| ID|Product Name| Price|Total Value|          sale|
+---+------------+------+-----------+--------------+
|  1|      Jacket|872.19|    7849.71|872.19|7849.71|
|  2|       Jeans|865.39|    1730.78|865.39|1730.78|
|  3|         Hat|183.11|     549.33| 183.11|549.33|
|  4|     Sweater|305.15|      610.3|  305.15|610.3|
|  5|         Hat| 84.59|      845.9|   84.59|845.9|
|  6|       Socks|191.72|    1533.76|191.72|1533.76|
|  7|      Jacket|696.02|     696.02| 696.02|696.02|
|  8|     Sweater|170.31|    1532.79|170.31|1532.79|
|  9|       Jeans|163.04|    1141.28|163.04|1141.28|
| 10|     Sweater| 990.3|     1980.6|  990.3|1980.6|
| 11|     Sweater| 92.88|     743.04|  92.88|743.04|
| 12|       Shoes|306.33|    1531.65|306.33|1531.65|
| 13|       Shirt| 65.67|     197.01|  65.67|197.01|
| 14|      Jacket|555.63|    4445.04|555.63|4445.04|
| 15|       Jeans| 48.99|     342.93|  48.99|342.93|
| 16|      Jacket|323.87|    2590.96|323.87|25

In [13]:
df4 = df3.select("ID","Product Name", explode(split(col("sale"), "\|")))
df4.show()

+---+------------+-------+
| ID|Product Name|    col|
+---+------------+-------+
|  1|      Jacket| 872.19|
|  1|      Jacket|7849.71|
|  2|       Jeans| 865.39|
|  2|       Jeans|1730.78|
|  3|         Hat| 183.11|
|  3|         Hat| 549.33|
|  4|     Sweater| 305.15|
|  4|     Sweater|  610.3|
|  5|         Hat|  84.59|
|  5|         Hat|  845.9|
|  6|       Socks| 191.72|
|  6|       Socks|1533.76|
|  7|      Jacket| 696.02|
|  7|      Jacket| 696.02|
|  8|     Sweater| 170.31|
|  8|     Sweater|1532.79|
|  9|       Jeans| 163.04|
|  9|       Jeans|1141.28|
| 10|     Sweater|  990.3|
| 10|     Sweater| 1980.6|
+---+------------+-------+
only showing top 20 rows



In [21]:
df4.select("Product Name").groupBy("ID").agg(collect_list(col("col")).alias("sale")).orderBy("ID").show()

AnalysisException: cannot resolve '`ID`' given input columns: [Product Name];
'Aggregate ['ID], ['ID, collect_list('col, 0, 0) AS sale#2772]
+- Project [Product Name#17]
   +- Project [ID#16, Product Name#17, col#226]
      +- Generate explode(split(sale#165, \|, -1)), false, [col#226]
         +- Project [ID#16, Product Name#17, Price#18, Total Value#19, concat_ws(|, cast(Price#18 as string), cast(Total Value#19 as string)) AS sale#165]
            +- Relation[ID#16,Product Name#17,Price#18,Total Value#19] csv


In [24]:
from pyspark.sql.functions import collect_list

df4.groupBy("ID").agg(collect_list("Product Name").alias("sale")).orderBy("ID").show()


+---+------------------+
| ID|              sale|
+---+------------------+
|  1|  [Jacket, Jacket]|
|  2|    [Jeans, Jeans]|
|  3|        [Hat, Hat]|
|  4|[Sweater, Sweater]|
|  5|        [Hat, Hat]|
|  6|    [Socks, Socks]|
|  7|  [Jacket, Jacket]|
|  8|[Sweater, Sweater]|
|  9|    [Jeans, Jeans]|
| 10|[Sweater, Sweater]|
| 11|[Sweater, Sweater]|
| 12|    [Shoes, Shoes]|
| 13|    [Shirt, Shirt]|
| 14|  [Jacket, Jacket]|
| 15|    [Jeans, Jeans]|
| 16|  [Jacket, Jacket]|
| 17|  [Jacket, Jacket]|
| 18|    [Shoes, Shoes]|
| 19|        [Hat, Hat]|
| 20|    [Socks, Socks]|
+---+------------------+
only showing top 20 rows

